In [79]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [80]:
file = "..\\data"+ "\\phaseone-tradedata-C-october.parquet"

df = pq.read_table(file).to_pandas()

df.replace({'Electrical equipment and mac.hioery':'Electrical equipment and machinery',
           'Industrial mac.hin ery': 'Industrial machinery'}, inplace = True)

df["hs2"] = df.hs4.str[0:2]

In [81]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

end_use = "hs?get=E_COMMODITY,ALL_VAL_YR,E_COMMODITY_SDESC"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time=2019" + "&COMM_LVL=HS2"

url = url + "&CTY_CODE=5700"

r = requests.get(url) 
    
print(r)
    
dfc = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

dfc.columns = r.json()[0]

dfc = dfc.groupby("E_COMMODITY").agg({"E_COMMODITY_SDESC":"first"})

<Response [200]>


In [82]:
dfc.head()

,E_COMMODITY_SDESC
E_COMMODITY,
01,LIVE ANIMALS
02,MEAT AND EDIBLE MEAT OFFAL
03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES"
04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...
05,"PRODUCTS OF ANIMAL ORIGIN, NESOI"


In [83]:
df.head()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge,hs2
0,2013-01-01,5700,4806.0,841090,0.0,-,8410,8410,"Hydraulic turbines, water wheels and regulator...",Industrial machinery,1. Manufactured Goods,both,84
1,2013-01-01,5700,287000.0,481031,0.0,-,4810,None,None,None,not in aggreement,left_only,48
2,2013-01-01,5700,311650.0,481160,0.0,-,4811,None,None,None,not in aggreement,left_only,48
3,2013-01-01,5700,63701.0,481620,0.0,-,4816,None,None,None,not in aggreement,left_only,48
4,2013-01-01,5700,40265.0,490290,0.0,-,4902,None,None,None,not in aggreement,left_only,49


In [84]:
aws = df.groupby(["hs2","time"]).agg({"china_exports":"sum"})

In [85]:
aws = aws.merge(dfc, left_on = ["hs2"], right_index = True, how = "left")

aws.reset_index(inplace = True)

aws["E_COMMODITY_SDESC"] = aws["E_COMMODITY_SDESC"] + ", HS CODE " + aws.hs2

In [86]:
aws.set_index(["E_COMMODITY_SDESC","time"],inplace = True)

In [87]:
out_file = ".\\data"+ "\\data.parquet"

pq.write_table(pa.Table.from_pandas(aws), out_file)

In [88]:
file = ".\\data"+ "\\data.parquet"

df = pq.read_table(file).to_pandas()

df.loc["IORN AND STEEL, HS CODE 72"]

hs2  \
E_COMMODITY_SDESC                                  time             
SPECIAL CLASSIFICATION PROVISIONS, NESOI, HS CO... 2020-05-01  98   
                                                   2020-06-01  98   
                                                   2020-07-01  98   
                                                   2020-08-01  98   
                                                   2020-09-01  98   

                                                               china_exports  
E_COMMODITY_SDESC                                  time                       
SPECIAL CLASSIFICATION PROVISIONS, NESOI, HS CO... 2020-05-01    100290089.0  
                                                   2020-06-01     90235038.0  
                                                   2020-07-01     85524371.0  
                                                   2020-08-01    102906355.0  
                                                   2020-09-01    104962569.0

In [89]:
df.china_exports.max()

3556098531.0

In [95]:
df[df.hs2 == "61"]

hs2  \
E_COMMODITY_SDESC                                  time             
APPAREL ARTICLES AND ACCESSORIES, KNIT OR CROCH... 2013-01-01  61   
                                                   2013-02-01  61   
                                                   2013-03-01  61   
                                                   2013-04-01  61   
                                                   2013-05-01  61   
...                                                            ..   
                                                   2020-05-01  61   
                                                   2020-06-01  61   
                                                   2020-07-01  61   
                                                   2020-08-01  61   
                                                   2020-09-01  61   

                                                               china_exports  
E_COMMODITY_SDESC                                  time                       
APPAREL ARTICLES AND ACCESSORIES, KNIT OR CROCH... 2013-01-01      1708428.0  
                                                   2013-02-01      1808692.0  
                                                   2013-03-01      2022087.0  
                                                   2013-04-01      1486485.0  
                                                   2013-05-01      2207350.0  
...                                                                      ...  
                                                   2020-05-01      5128153.0  
                                                   2020-06-01      3810085.0  
                                                   2020-07-01      4410275.0  
                                                   2020-08-01      4255356.0  
                                                   2020-09-01      4346943.0  

[93 rows x 2 columns]